In [1]:
import json
import numpy as np

import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('sentiwordnet')

from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from afinn import Afinn
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import custom_lexicons.senticnet.senticnet as sentic

import numpy as np 
import pandas as pd
import data_reader
import results_analyser
import itertools

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\phoec\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\phoec\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     C:\Users\phoec\AppData\Roaming\nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


In [2]:
FILE_DIR = "./custom_lexicons/"

NTUSD_FILEPATH = "ntusd/NTUSD_Fin_word_v1.0.json"
STOCKTWITLEXI_FILEPATH = "stocktwitlexi/domain_lexicon_raw_norm.csv"
SENTI_DD_FILEPATH = "sentidd/sentidd_data1.csv"
LM_FILEPATH = "sentidd/LM_Word_List.csv"

# AFINN
afinn = Afinn()

# VADER
analyzer = SentimentIntensityAnalyzer()

# NTUSD-FIN
with open(FILE_DIR+NTUSD_FILEPATH, "r") as f:
    data = f.read()
    NTUSD = json.loads(data)
word_sent_dict = {}
for i in range(len(NTUSD)):
    word_sent_dict[NTUSD[i]["token"]] = NTUSD[i]["market_sentiment"]
    
# STOCKTWITLEXI    
stocktwitlexi = pd.read_csv(FILE_DIR+STOCKTWITLEXI_FILEPATH, header=None, index_col=0)
stocktwitlexi = stocktwitlexi.to_dict()[1]

# SENTIDD
sentidd = pd.read_csv(FILE_DIR+SENTI_DD_FILEPATH)
sentidd_dict = dict(zip(zip(sentidd.entity, sentidd.directional_word), sentidd.sentiment))
lm_df = pd.read_csv(FILE_DIR+LM_FILEPATH)
lm_dict = dict(zip(lm_df.word, lm_df.label))

stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def remove_stopwords(data):
    sentence_token = [s.split(' ') for s in data] 
    return sentence_token

def get_wordnet_tag(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

In [3]:
def standardise_scores(score, count):
    if (count>0):
        return score/count
    else:
        return score

def normalise_scores(pred_raw, lexicon):
    if(lexicon=="senticnet"): #-1 to 1
        return pred_raw
    elif(lexicon=="ntusd"): #-3.81 to 1.22, range is 5 so /2.5
        return [pred/2.5 for pred in pred_raw]
    elif(lexicon=="sentiwordnet"): #-1 to 1 since pos-neg
        return pred_raw
    elif(lexicon=="stocktwitlexi"): #-1 to 1
        return pred_raw
    elif(lexicon=="afinn"): #-5 to 5
        return [pred/5 for pred in pred_raw]
    elif(lexicon=="vader"): #-1 to 1
        return pred_raw
    elif(lexicon=="sentidd"): #-2 to 2
        return [pred/2 for pred in pred_raw]

In [4]:
def senti_dd_polarity(text, sentidd_dict, lm_dict):
    def lm_score(text, lm_dict):
        tokens = word_tokenize(text)
        count = 0
        score = 0
        for token in tokens:
            try:
                if lm_dict[token]=="positive":
                    score += 1
                    count += 1
                elif lm_dict[token]=="negative":
                    score -= 1
                    count += 1
            except:
                pass

        return score/count if count>0 else score

    def senti_dd_score(text, sentidd_dict):
        tokens = word_tokenize(text)
        count = 0
        score = 0
        stemmed_tokens = [stemmer.stem(token) for token in tokens]
        lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
        for stemmed_token in stemmed_tokens:
            for lemmatized_token in lemmatized_tokens:
                
                try:
                    if (sentidd_dict[(lemmatized_token, stemmed_token)] =="positive"):
                        score += 1
                        count += 1
                    elif (sentidd_dict[(lemmatized_token, stemmed_token)] =="negative"):
                        score -= 1
                        count += 1
                except:
                    pass
        return score/count if count>0 else score
    
    score = lm_score(text, lm_dict)
    context_sentiment_score = senti_dd_score(text, sentidd_dict)
    if context_sentiment_score > 0: score += 1
    elif context_sentiment_score < 0: score -= 1

    return score

In [5]:
def individual_scoring(X, y_class):
    sentic_pred_raw = []
    ntusd_pred_raw = []
    sentiwordnet_pred_raw = []
    stocktwitlexi_pred_raw = []
    afinn_pred_raw = []
    vader_pred_raw = []
    sentidd_pred_raw = []
    
    for test_tweet in X:
        sentic_score = 0
        ntusd_score = 0
        sentiwordnet_score = 0
        stocktwitlexi_score = 0
        afinn_score = 0
        
        sentic_count = 0
        ntusd_count = 0
        sentiwordnet_count = 0
        stocktwitlexi_count = 0
        afinn_count = 0
        
        sentence_tagged = np.array(nltk.pos_tag(test_tweet))
        for tagged in sentence_tagged:
            word = tagged[0]
            wn_tag = get_wordnet_tag(tagged[1])
            
            #Senticnet
            try: 
                sentic_score += sentic.senticnet[word][7]
                sentic_count += 1
            except:
                pass
            #NTUSD
            try: 
                ntusd_score += word_sent_dict[word]
                ntusd_count += 1
            except:
                pass
            
            #Senticwordnet
            if wn_tag in (wn.NOUN, wn.ADJ, wn.ADV,  wn.VERB):            
                lemma = lemmatizer.lemmatize(word, pos=wn_tag)
                if lemma:
                    synsets = wn.synsets(lemma, pos=wn_tag)
                    if synsets:
                        swn_synset = swn.senti_synset(synsets[0].name())
                        sentiwordnet_score += swn_synset.pos_score() - swn_synset.neg_score()
                        sentiwordnet_count += 1
            #Stocktwitlexi
            try: 
                stocktwitlexi_score += stocktwitlexi[word]
                stocktwitlexi_count += 1
            except:
                pass
            
            #Afinn
            try: 
                afinn_score += afinn.score(word)
                afinn_count += 1
            except:
                pass
        
        #Afinn
        s = " ".join(test_tweet)
        
        #Vader
        vader_output = analyzer.polarity_scores(s)
        vader_score = vader_output["compound"] #vader_output["pos"] - vader_output["neg"]
        
        #Senti-DD
        sentidd_score = senti_dd_polarity(s, sentidd_dict, lm_dict)
        
        sentic_score = standardise_scores(sentic_score, sentic_count)
        ntusd_score = standardise_scores(ntusd_score, ntusd_count)
        sentiwordnet_score = standardise_scores(sentiwordnet_score, sentiwordnet_count)
        stocktwitlexi_score = standardise_scores(stocktwitlexi_score, stocktwitlexi_count)
        afinn_score = standardise_scores(afinn_score, afinn_count)
        vader_score = vader_score #already normalised
        sentidd_score = sentidd_score #already normalised
        
        sentic_pred_raw.append(sentic_score)
        ntusd_pred_raw.append(ntusd_score)
        sentiwordnet_pred_raw.append(sentiwordnet_score)
        stocktwitlexi_pred_raw.append(stocktwitlexi_score)
        afinn_pred_raw.append(afinn_score)
        vader_pred_raw.append(vader_score)
        sentidd_pred_raw.append(sentidd_score)
        
    sentic_pred = normalise_scores(sentic_pred_raw, "senticnet")
    ntusd_pred = normalise_scores(ntusd_pred_raw, "ntusd")
    sentiwordnet_pred = normalise_scores(sentiwordnet_pred_raw, "sentiwordnet")
    stocktwitlexi_pred = normalise_scores(stocktwitlexi_pred_raw, "stocktwitlexi")
    afinn_pred = normalise_scores(afinn_pred_raw, "afinn")
    vader_pred = normalise_scores(vader_pred_raw, "vader")
    sentidd_pred = normalise_scores(sentidd_pred_raw, "sentidd")

    combined_df = pd.DataFrame()
    combined_df['senticnet'] = sentic_pred
    combined_df['ntusd'] = ntusd_pred
    combined_df['sentiwordnet'] = sentiwordnet_pred
    combined_df['afinn'] = afinn_pred
    combined_df['vader'] = vader_pred
    combined_df['stocktwitlexi'] = stocktwitlexi_pred
    combined_df['sentidd'] = sentidd_pred
    combined_df['actual_class'] = y_class
    
    return combined_df

In [6]:
def combine_voting_leave2soft(row):
    return row.sort_values().iloc[1:4].mean()

In [7]:
def generate_results(combined_df):
    combination_list = []
    lexicons = ["senticnet", "ntusd", "sentiwordnet", "stocktwitlexi", "afinn", "vader", "sentidd"]

    for i in range(1, len(lexicons)+1):
        combination_tuples = list(itertools.combinations(lexicons, i))
        combination_list.extend([list(elem) for elem in combination_tuples])

    y_class = combined_df['actual_class']
    results_df = pd.DataFrame()

    for combination in combination_list:
        curr_df = combined_df[combination]

        if(len(combination)>1): #more than 1
            voting_soft = curr_df.apply(combine_voting_soft, axis=1)
            pred_class = results_analyser.probability_to_class(voting_soft)
            results_df = results_analyser.calculate_metrics(results_df, y_class, pred_class, '_'.join(combination)+"_soft")

            if(len(combination)%2==1): #odd number
                voting_hard = curr_df.apply(combine_voting_hard, axis=1)
                results_df = results_analyser.calculate_metrics(results_df, y_class, voting_hard, '_'.join(combination)+"_hard")

            if(len(combination)>=5 and len(combination)%2==1): #odd and gte 5
                voting_leave2soft = curr_df.apply(combine_voting_leave2soft, axis=1)
                pred_class = results_analyser.probability_to_class(voting_leave2soft)
                results_df = results_analyser.calculate_metrics(results_df, y_class, pred_class, '_'.join(combination)+"_leave2soft")
        else:
            pred_class = results_analyser.probability_to_class(combined_df[combination[0]])
            results_df = results_analyser.calculate_metrics(results_df, y_class, pred_class, '_'.join(combination))

    return results_df

## Post analysis

### Read in Data2 and 3, combine them both

In [8]:
data2_X, data2_y_class = data_reader.read_data2("list")
data3_X, data3_y_class = data_reader.read_data3("list")

data2_X.extend(data3_X)
data2_y_class.extend(data3_y_class)

In [9]:
len(data2_X)

2036

### Get individual Scores

In [10]:
combined_df = individual_scoring(data2_X, data2_y_class)
senticnet_finegrained = combined_df['senticnet']
combined_df

,senticnet,ntusd,sentiwordnet,afinn,vader,stocktwitlexi,sentidd,actual_class
0,0.302600,-0.084072,0.015625,0.038095,0.5859,0.062336,0.5,1
1,-0.126786,0.131360,0.017045,-0.005263,0.2023,0.088886,-0.5,0
2,0.137750,0.026968,-0.013158,-0.024242,-0.7351,0.088475,0.0,1
3,0.857500,0.286495,0.020833,0.018182,0.3818,0.089604,0.0,1
4,0.854500,0.309721,0.015625,0.040000,0.6249,0.116167,0.5,1
...,...,...,...,...,...,...,...,...
2031,0.446667,-0.015374,-0.093750,-0.027273,-0.0783,0.095814,-0.5,0
2032,0.024500,-0.425183,-0.062500,0.000000,0.0000,0.088961,0.0,0
2033,0.472500,-0.472243,0.000000,-0.100000,-0.3182,-0.134641,0.0,0
2034,0.879000,-0.260604,0.000000,0.000000,0.0000,0.090518,0.0,0


### Test best strategy without vader, compare usefulness
Best strategy: NTUSD-Fin, StockTwitLexi, Afinn, Vader and Senti-DD 

In [11]:
combined_df = combined_df[['ntusd', 'stocktwitlexi', 'afinn', 'vader', 'sentidd', 'actual_class']]
combined_df

,ntusd,stocktwitlexi,afinn,vader,sentidd,actual_class
0,-0.084072,0.062336,0.038095,0.5859,0.5,1
1,0.131360,0.088886,-0.005263,0.2023,-0.5,0
2,0.026968,0.088475,-0.024242,-0.7351,0.0,1
3,0.286495,0.089604,0.018182,0.3818,0.0,1
4,0.309721,0.116167,0.040000,0.6249,0.5,1
...,...,...,...,...,...,...
2031,-0.015374,0.095814,-0.027273,-0.0783,-0.5,0
2032,-0.425183,0.088961,0.000000,0.0000,0.0,0
2033,-0.472243,-0.134641,-0.100000,-0.3182,0.0,0
2034,-0.260604,0.090518,0.000000,0.0000,0.0,0


In [12]:
combined_df['leave2soft_w_vader'] = combined_df.loc[:, combined_df.columns != 'actual_class'].apply(combine_voting_leave2soft, axis=1)
combined_df['leave2soft_wo_vader'] = combined_df.loc[:, ~combined_df.columns.isin(['actual_class', 'vader'])].apply(combine_voting_leave2soft, axis=1)

combined_df

C:\Users\phoec\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,ntusd,stocktwitlexi,afinn,vader,sentidd,actual_class,leave2soft_w_vader,leave2soft_wo_vader
0,-0.084072,0.062336,0.038095,0.5859,0.5,1,0.200144,0.100192
1,0.131360,0.088886,-0.005263,0.2023,-0.5,0,0.071661,0.051761
2,0.026968,0.088475,-0.024242,-0.7351,0.0,1,0.000909,0.009292
3,0.286495,0.089604,0.018182,0.3818,0.0,1,0.131427,0.079738
4,0.309721,0.116167,0.040000,0.6249,0.5,1,0.308629,0.244839
...,...,...,...,...,...,...,...,...
2031,-0.015374,0.095814,-0.027273,-0.0783,-0.5,0,-0.040315,-0.027654
2032,-0.425183,0.088961,0.000000,0.0000,0.0,0,0.000000,0.000000
2033,-0.472243,-0.134641,-0.100000,-0.3182,0.0,0,-0.184280,-0.139640
2034,-0.260604,0.090518,0.000000,0.0000,0.0,0,0.000000,0.000000


### Add text field

In [13]:
combined_df['text'] = [" ".join(l) for l in data2_X]

### Convert fine-grained score to class predictions

In [14]:
y_class = combined_df['actual_class']
combined_df['leave2soft_w_vader_class'] = results_analyser.probability_to_class(combined_df['leave2soft_w_vader'])
combined_df['leave2soft_wo_vader_class'] = results_analyser.probability_to_class(combined_df['leave2soft_wo_vader'])
combined_df = combined_df[['vader', 'actual_class', 'leave2soft_w_vader', 'leave2soft_wo_vader', 'leave2soft_w_vader_class', 'leave2soft_wo_vader_class', 'text']]
combined_df

,vader,actual_class,leave2soft_w_vader,leave2soft_wo_vader,leave2soft_w_vader_class,leave2soft_wo_vader_class,text
0,0.5859,1,0.200144,0.100192,1,1,rt yo enter to win monarch tokens us stock mar...
1,0.2023,0,0.071661,0.051761,1,1,srilanka surcharge on fuel removed the surchar...
2,-0.7351,1,0.000909,0.009292,1,1,net issuance increases to fund fiscal programs...
3,0.3818,1,0.131427,0.079738,1,1,rt how much of amazons traffic is served by fa...
4,0.6249,1,0.308629,0.244839,1,1,ryzen desktop cpus looking great and on track ...
...,...,...,...,...,...,...,...
2031,-0.0783,0,-0.040315,-0.027654,0,0,while all other cryptos are rising and shining...
2032,0.0000,0,0.000000,0.000000,0,0,glad i sold this crap hbar
2033,-0.3182,0,-0.184280,-0.139640,0,0,bull trap
2034,0.0000,0,0.000000,0.000000,0,0,team bear


### Find instances where our best strategy predicted correctly

In [15]:
vader_correct_df = combined_df.loc[(combined_df['leave2soft_w_vader_class'] == combined_df['actual_class'])]
vader_correct_df

,vader,actual_class,leave2soft_w_vader,leave2soft_wo_vader,leave2soft_w_vader_class,leave2soft_wo_vader_class,text
0,0.5859,1,0.200144,0.100192,1,1,rt yo enter to win monarch tokens us stock mar...
2,-0.7351,1,0.000909,0.009292,1,1,net issuance increases to fund fiscal programs...
3,0.3818,1,0.131427,0.079738,1,1,rt how much of amazons traffic is served by fa...
4,0.6249,1,0.308629,0.244839,1,1,ryzen desktop cpus looking great and on track ...
5,0.3818,1,0.065108,0.059303,1,1,rt reduce your portfolio risk gold is a perfec...
...,...,...,...,...,...,...,...
2030,0.0258,0,-0.144183,-0.192244,0,0,silly bulls
2031,-0.0783,0,-0.040315,-0.027654,0,0,while all other cryptos are rising and shining...
2032,0.0000,0,0.000000,0.000000,0,0,glad i sold this crap hbar
2033,-0.3182,0,-0.184280,-0.139640,0,0,bull trap


### Find instances where our best strategy predicted correctly & strategy w/o vader predicted wrongly

In [16]:
pd.set_option('display.max_colwidth', None)

In [17]:
vader_correct_df_contrast = vader_correct_df.loc[(vader_correct_df['leave2soft_wo_vader_class'] != vader_correct_df['actual_class'])]
vader_correct_df_contrast

,vader,actual_class,leave2soft_w_vader,leave2soft_wo_vader,leave2soft_w_vader_class,leave2soft_wo_vader_class,text
21,-0.1531,0,-0.032115,0.008214,0,1,rt several large corporations are stopping advertising on facebook either till july or for the whole of this year if these
35,-0.2023,0,-0.046601,0.005298,0,1,rt biggest mistakes of the century from before apple is only a pc maker amazon is only a bookstore goo
213,-0.1531,0,-0.032115,0.008214,0,1,rt several large corporations are stopping advertising on facebook either till july or for the whole of this year if these
245,-0.3400,0,-0.028465,0.006491,0,1,crazy opening gaps
255,-0.6486,0,-0.004048,0.001849,0,1,unless you are an arrogant letter writeradvice seller not all are but some are it is time to realize that the obvious problems in this economy and the financial mkts provide an uncertain road w both risk and opportunity even if u are a pious shark
278,0.5106,1,0.005451,-0.013879,1,0,rt the on whats driving the shorting of stocks that has seen bets against the spdr sp trust the biggest exchange
352,-0.8625,0,-0.000514,0.020148,0,1,rt saturday country descends into chaos sunday chaos deepens monday rallies for no reason trump declares
405,-0.1027,0,-0.018432,0.009658,0,1,hope not i added some during the panic selloff
438,-0.8750,0,-0.005531,0.030720,0,1,rt be greedy when others are fearful and be fearful when others are greedy
441,0.3818,1,0.002611,-0.046279,1,0,rt in made new all time highs on declining earnings growth in made new all time highs on declining


### Get the words processed from this df

In [18]:
processed_words_list = []

for row in vader_correct_df_contrast.text.tolist():
    processed_words = []
    tokens = row.split(" ")
    for token in tokens:
        vader_score = analyzer.polarity_scores(token)["compound"]
        if(vader_score!=0):
            processed_words.append(token)
          
    processed_words_list.append(processed_words)

print(processed_words_list)

[['stopping'], ['mistakes', 'amazon'], ['stopping'], ['crazy'], ['arrogant', 'problems', 'uncertain', 'risk', 'opportunity'], ['trust'], ['chaos', 'chaos', 'no'], ['hope', 'panic'], ['greedy', 'fearful', 'fearful', 'greedy'], ['growth'], ['racism'], ['miss'], ['crash', 'crash', 'truth', 'stop', 'fighting'], ['suspended', 'forced'], ['dumping'], ['no'], ['growth', 'worst', 'original'], ['confusing'], ['honest'], ['clarity', 'failing'], ['good'], ['reached'], ['dump'], ['shares', 'share', 'creation', 'lack'], ['doom', 'gloom'], ['dumped', 'lol'], ['rich', 'prepared', 'destroyed'], ['miss'], ['ready', 'want', 'miss'], ['no', 'credit'], ['no', 'fear'], ['ok'], ['speculative', 'asset', 'scams', 'shitty', 'agree', 'good', 'dead'], ['screwed']]


In [21]:
vader_correct_df_contrast['processed_words'] = processed_words_list
vader_correct_df_contrast[['actual_class', 'leave2soft_w_vader', 'leave2soft_wo_vader', 'text', 'processed_words', 'vader']].reset_index(drop=True)

C:\Users\phoec\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,actual_class,leave2soft_w_vader,leave2soft_wo_vader,text,processed_words,vader
0,0,-0.032115,0.008214,rt several large corporations are stopping advertising on facebook either till july or for the whole of this year if these,[stopping],-0.1531
1,0,-0.046601,0.005298,rt biggest mistakes of the century from before apple is only a pc maker amazon is only a bookstore goo,"[mistakes, amazon]",-0.2023
2,0,-0.032115,0.008214,rt several large corporations are stopping advertising on facebook either till july or for the whole of this year if these,[stopping],-0.1531
3,0,-0.028465,0.006491,crazy opening gaps,[crazy],-0.3400
4,0,-0.004048,0.001849,unless you are an arrogant letter writeradvice seller not all are but some are it is time to realize that the obvious problems in this economy and the financial mkts provide an uncertain road w both risk and opportunity even if u are a pious shark,"[arrogant, problems, uncertain, risk, opportunity]",-0.6486
5,1,0.005451,-0.013879,rt the on whats driving the shorting of stocks that has seen bets against the spdr sp trust the biggest exchange,[trust],0.5106
6,0,-0.000514,0.020148,rt saturday country descends into chaos sunday chaos deepens monday rallies for no reason trump declares,"[chaos, chaos, no]",-0.8625
7,0,-0.018432,0.009658,hope not i added some during the panic selloff,"[hope, panic]",-0.1027
8,0,-0.005531,0.030720,rt be greedy when others are fearful and be fearful when others are greedy,"[greedy, fearful, fearful, greedy]",-0.8750
9,1,0.002611,-0.046279,rt in made new all time highs on declining earnings growth in made new all time highs on declining,[growth],0.3818


In [22]:
vader_correct_df_contrast[['actual_class', 'leave2soft_w_vader', 'leave2soft_wo_vader', 'text', 'processed_words', 'vader']].iloc[[6,7,14,22,33]].reset_index(drop=True)

,actual_class,leave2soft_w_vader,leave2soft_wo_vader,text,processed_words,vader
0,0,-0.000514,0.020148,rt saturday country descends into chaos sunday chaos deepens monday rallies for no reason trump declares,"[chaos, chaos, no]",-0.8625
1,0,-0.018432,0.009658,hope not i added some during the panic selloff,"[hope, panic]",-0.1027
2,0,-0.041898,0.016542,making a comeback and dumping up points in that now,[dumping],-0.3182
3,0,-0.005511,0.006697,once nothing is fixed by the third hike its gonna be a bigger basis point for the next and market gonna dump already planned in advance i bet,[dump],-0.3818
4,0,-0.016744,0.011008,china screwed us all,[screwed],-0.4939
